In [1]:
# 예제 1. 일반적으로 Decorator는 함수를 다른 함수로 대체한다

def deco(func) :
  def inner() :
    print('running inner()')
  return inner

In [2]:
@deco
def target():
  print("running target()")

target()

running inner()


In [3]:
target

<function __main__.deco.<locals>.inner()>

In [4]:
# 예제 2. Decorator를 실행하는 시점 예제

registry = []

def register(func) :
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
  print('running main()')
  print('registry ->', registry)
  f1()
  f2()
  f3()

if __name__ == '__main__':
  main()

running register(<function f1 at 0x7ebed2ece440>)
running register(<function f2 at 0x7ebf102101f0>)
running main()
registry -> [<function f1 at 0x7ebed2ece440>, <function f2 at 0x7ebf102101f0>]
running f1()
running f2()
running f3()


In [5]:
# 예제 3. Promotion decorator로 채운 promos list

promos = []

def promotion(promo_func):
  """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
  promos.append(promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

@promotion
def large_order_promo(order) :
  """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

def best_promo(order) :
  """최대로 할인받을 금액을 반환한다."""
  return max(promo(order) for promo in promos)

In [6]:
# 예제 4. 지역 및 전역 변수를 읽는 함수

def f1(a):
  print(a)
  print(b)

f1(3)

3


NameError: name 'b' is not defined

In [7]:
b=6
f1(3)

3
6


In [8]:
# 예제 5. 함수 본체 안에서 값을 할당하기 때문에 지역변수가 되는 b

b = 6
def f2(a):
  print(a)
  print(b)
  b = 9

f2(3)


3


UnboundLocalError: local variable 'b' referenced before assignment

In [9]:
# 예제 6. global 선언 후 변화

b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9

f3(3)

3
6


In [10]:
b

9

In [11]:
f3(3)

3
9


In [12]:
b = 30
f3(3)

3
30


In [13]:
# 예제 7. 이동 평균을 계산하는 클래스

class Averager():
  def __init__(self):
    self.series = []

  def __call__(self, new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [14]:
avg = Averager()
avg(10)

10.0

In [15]:
avg(11)

10.5

In [16]:
avg(12)

11.0

In [17]:
# 예제 8. 이동 평균을 계산하는 high-order function

def make_averager():
  series = []
  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)

  return averager


In [18]:
avg = make_averager()
avg(10)

10.0

In [19]:
avg(11)

10.5

In [20]:
avg(12)

11.0

In [21]:
# 예제 9. make_averager()로 생성한 함수 조사하기

avg.__code__.co_varnames

('new_value', 'total')

In [22]:
avg.__code__.co_freevars

('series',)

In [23]:
avg.__closure__

(<cell at 0x7ebed2f1a350: list object at 0x7ebed2dd5ec0>,)

In [24]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [25]:
# 예제 10. 전체 이력을 유지하지 않고 이동 평균을 잘못 계산하는 잘못된 high-order function

def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total / count

  return averager

In [26]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [27]:
# 예제 11. 전체 이력을 유지하지 않고 이동 평균 계산하기 (nonlocal로 추정)

def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count

  return averager

In [28]:
avg = make_averager()
avg(10)

10.0

In [29]:
# 예제 12. 함수의 실행 시간을 출력하는 간단한 decorator

import time

def clock(func) :
  def clocked(*args) :
    t0 = time.perf_counter()
    result = func(*args)
    elapsed = time.perf_counter() - t0
    name = func.__name__
    arg_str = ', '.join(repr(arg) for arg in args)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
    return result
  return clocked

In [30]:
# 예제 13. clock decorator 사용하기

@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! = ', factorial(6))

**************************************** Calling snooze(.123)
[0.12322604s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000070s] factorial(1) -> 1
[0.00003280s] factorial(2) -> 2
[0.00193352s] factorial(3) -> 6
[0.00199347s] factorial(4) -> 24
[0.00201908s] factorial(5) -> 120
[0.00204674s] factorial(6) -> 720
6! =  720


In [31]:
# 예제 14. 개선된 clock decorator

import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked(*args,**kwargs):
    t0 = time.time()
    result = func(*args,**kwargs)
    elapsed = time.time() - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed,name,arg_str,result))
    return result
  return clocked

In [32]:
@clock
def snooze(seconds):
  time.sleep(seconds)

@clock
def factorial(n) :
  return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12320638s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00040770s] factorial(2) -> 2
[0.00077605s] factorial(3) -> 6
[0.00115657s] factorial(4) -> 24
[0.00153446s] factorial(5) -> 120
[0.00189638s] factorial(6) -> 720
6! = 720


In [33]:
factorial.__name__

'factorial'